# **DSFM Project**: Customer Segmentation - K-means Clustering (SOLUTIONS)

Creator: [Data Science for Managers - EPFL Program](https://www.dsfm.ch)  
Source:  [https://github.com/dsfm-org/code-bank.git](https://github.com/dsfm-org/code-bank.git)  
License: [MIT License](https://opensource.org/licenses/MIT). See open source [license](LICENSE) in the Code Bank repository. 

------

## Overview

<img src="https://conversionxl.com/wp-content/uploads/2016/09/segmentation-illustration.png" width="500" height="500" align="center"/>

Image source: https://conversionxl.com/wp-content/uploads/2016/09/segmentation-illustration.png

Dataset source: *Abreu, N. (2011). Analise do perfil do cliente Recheio e desenvolvimento de um sistema promocional. Mestrado em Marketing, ISCTE-IUL, Lisbon* 

The goal of this demo is to cluster customers of a wholesale Portugese company into different segments. Note that clustering is an unsupervised learning task where no labels are given. Description of the fields are as follows (m.u. stands for monetary unit): 


| Feature name     | Variable Type | Description 
|------------------|---------------|--------------------------------------------------------
| FRESH            | Continuous    | annual spending (m.u.) on fresh products   
| MILK             | Continuous    | annual spending (m.u.) on milk products  
| GROCERY          | Continuous    | annual spending (m.u.) on grocery products  
| FROZEN           | Continuous    | annual spending (m.u.) on frozen products   
| DETERGENTS_PAPER | Continuous    | annual spending (m.u.) on detergents and paper products  
| DELICATESSEN     | Continuous    | annual spending (m.u.) on delicatessen products    
| CHANNEL          | Categorical   | customers channel where 1 = HoReCa (Hotel/Restaurant/Cafe); 2 = Retail channel
| REGION           | Categorical   | customers region where 1 = Lisbon; 2 = Porto; 3 = Other  

------

## **Part 0**: Setup

In [ ]:
# Put all import statements at the top of your notebook

# Standard imports
import pandas as pd
import numpy  as np
from time import time

# Clustering imports
from sklearn               import preprocessing
from sklearn               import datasets
from sklearn.cluster       import KMeans
from sklearn.metrics       import silhouette_samples, silhouette_score
from sklearn.decomposition import PCA
from sklearn.manifold      import TSNE

# Visualization packages
from bokeh.models         import HoverTool
from bokeh.plotting       import output_notebook, figure, show, ColumnDataSource
import bokeh.plotting     as bp
import matplotlib.pyplot  as plt
import matplotlib.cm      as cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker    import NullFormatter
from faerun               import Faerun

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


In [ ]:
# Set a seed for replication
SEED = 10

## **Part 1**: Data Preprocessing and EDA

First, we would like to understand the main characteristics of the dataset. We might need to transform and clean some features before we can specify a statistical model.

**Q 1:** Load the `customer_data.csv` file, check the data shape, and investigate the top 10 rows?

In [ ]:
# Load data
data = pd.read_csv('customer_data.csv')

In [ ]:
# Number of observations and features
data.shape

In [ ]:
# Top rows
data.head(10)

**Q 2:** Investigate the available features and their data types. Do they look OK?

In [ ]:
# Features
data.columns

In [ ]:
# Feature types
data.dtypes

**Q 3:** Compute descriptive statistics across all features. For the `Channel` and `Region` features, count how often the feature values occur. What's the most common Channel and Region?

Hint: Use the `value_counts()` function in Pandas to count feature values.

In [ ]:
# Key statistics
data.describe()

In [ ]:
data['Channel'].value_counts()

In [ ]:
data['Region'].value_counts()

The most common Channel is _HoReCa (Hotel/Restaurant/Cafe)_. The most common Region is _Other_, the regions outside of Lisbon and Porto.

## **Part 2**: Data Preprocessing

There are several preprocessing steps before doing the clustering:

* null values
* categorical variables
* standardization

**Q 1:** Check for redundant and missing values. How many duplicate rows and missing values are there? 

In [ ]:
# Drop possible duplicate rows
data.drop_duplicates(inplace=True)
data.shape

In [ ]:
# Drop possible missing values
data.dropna(inplace=True)
data.shape

There are no duplicates and missing values in our dataset.

**Q 2:** One-hot encode the categorical featues `Channel` and `Region`. What's the new shape of the dataset? Why?

In [ ]:
cols_to_transform = [ 'Channel', 'Region']
data_with_dummies = pd.get_dummies(data = data, columns = cols_to_transform )

In [ ]:
data_with_dummies.shape

In [ ]:
data_with_dummies.head()

**Q 3:** Standardize data such that each feature has a mean of zero and a standard deviation of one. Validate the standardization has worked by computing descriptive statistics on the standardized features.

In [ ]:
# Aside: check the underlying data 
X_scaled = preprocessing.scale(data_with_dummies.values)
data_with_dummies_ready = pd.DataFrame(X_scaled, columns = data_with_dummies.columns)

In [ ]:
data_with_dummies_ready.head()

In [ ]:
data_with_dummies_ready.describe()

## **Part 3**: k-means Clustering

We know cluster the customers into three clusters, using k-means clustering. Why would that be helpful? Think about a marketing campaign that targets customers with shared characteristics, e.g. customers in a particular region of Portugal ordering similar amounts of different foods.

**Q 1:** Fit a k-means clustering model with three clusters.

Hint: Don't forget using the `SEED` value to ensure consistent results across multiple runs. 

In [ ]:
# Clustering data into 3 clusters
kmeans = KMeans(n_clusters=3, random_state=SEED).fit(data_with_dummies_ready)

**Q 2:** What are the cluster centers/centroids and what do they represent? What clusters do the customers belong to?

Hint: Use the `cluster_centers_` and `labels_` attributes of the model trained above.

In [ ]:
kmeans.cluster_centers_

Each observation belongs to the cluster with the nearest mean, i.e. the centroids are a proxy for the entire cluster they belong to.

In [ ]:
data_with_clusters = data.copy()
data_with_clusters['Cluster_k=3'] = kmeans.labels_

data_with_clusters

We should pay attention to the following:

* __Initial centroids__: k-mean clustering in sklearn is initializing centoids with different initial random seeds and will report the best result in terms of optimization score.  
  
* __Number of clusters__: to select the best number of clusters, one strategy is to maximize the "Silhouette" coefficient.  
 

## **Part 4**: Selecting the optimal number of clusters, _k_

There are different approaches to selecting the number of clusters, _k_. One approach uses domain knowledge of the problem at hand. Another approach evaluates how well different values for _k_ segment the space. In this part, we will use the Silhouette coefficient to find the (computationally) optimal number of clusters.

The Silhouette coefficient is calculated using the mean distance of a data point to all other points in the same cluster (a) and the mean distance to all data points in the nearest-cluster (b). The Silhouette Coefficient for a sample is (b - a) / max(a, b). The best value is 1 and the worst value is -1. Values near 0 indicate overlapping clusters. Negative values generally indicate that a sample has been assigned to the wrong cluster, as a different cluster is more similar. We can consider the mean Silhuette coefficient of all samples as a rough metric for clustering quality.  

We change number of clusters from 2 to 20 and chose the one with best Silouette score. For each number of clusters, we visualize the silouette score of each data point and project our data using PCA into 2 dimensions to visualize detected clusters.

In [ ]:
X = data_with_dummies_ready
range_n_clusters = range(2,21)

# For different number of clusters do the followings
for n_clusters in range_n_clusters:
    
    # Define a subplot with one row and two columns
    fig, (ax1,ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)
    ax1.set_xlim([-1, 1])
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])
    
    # Obtain cluster labels for n_clusters
    clusterer = KMeans(n_clusters=n_clusters, random_state=SEED)
    cluster_labels = clusterer.fit_predict(X)
    
    # Calculate average silhouette score 
    silhouette_avg = silhouette_score(X, cluster_labels)

    # Calculate silhouette score for each data point
    sample_silhouette_values = silhouette_samples(X, cluster_labels)
    
    # Visualize silhouette score of each data point as well as average silhouette score
    y_lower = 10
    for i in range(n_clusters):
       
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
        ith_cluster_silhouette_values.sort()
        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))
        y_lower = y_upper + 10  

    ax1.set_title('The silhouette plot for the various clusters.')
    ax1.set_xlabel('The silhouette coefficient values')
    ax1.set_ylabel('Cluster label')
    ax1.axvline(x=silhouette_avg, color='red', linestyle='--')
    ax1.set_yticks([])  
    ax1.set_xticks([-1, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
    
    # Project data into two dimensions using PCA, with differnt colors for each cluster
    pca = PCA(n_components=2, svd_solver='full')
    XX = pca.fit(X).transform(X)
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(XX[:, 0], XX[:, 1], marker='.', s=30, lw=0, alpha=0.7, c=colors, edgecolor='k')
    centers = pca.transform(clusterer.cluster_centers_)
   
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',c="white", alpha=1, s=200, edgecolor='k')
    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1, s=50, edgecolor='k')

    ax2.set_title('The visualization of the clustered data.')
    ax2.set_xlabel('Feature space for the 1st feature')
    ax2.set_ylabel('Feature space for the 2nd feature')
    
    plt.suptitle(('\n Silhouette analysis for KMeans clustering on sample data '
                  'with n_clusters = %d (Avg score: %f)' % (n_clusters, silhouette_avg)), fontsize=14, fontweight='bold')

    plt.show()

In [ ]:
# Calculate the silhouette score with different number of clusters and different random seeds
range_seeds = range(0,10)
range_n_clusters = range(2,21)
list_all = []
list_cluster = []
for seed in range_seeds:
    for n_clusters in range_n_clusters:
        
        clusterer = KMeans(n_clusters=n_clusters, random_state=seed)
        cluster_labels = clusterer.fit_predict(data_with_dummies_ready)
        silhouette_avg = silhouette_score(data_with_dummies_ready, cluster_labels)
        
        list_cluster.append(silhouette_avg)
        list_cluster_tmp = list(list_cluster)
        
    list_all.append(list_cluster_tmp)
    list_cluster.clear()

# Average across all runs
gather_list_all = []
for i in range_seeds:
    gather_list_all.append(list_all[i])

results_avg = [float(sum(col))/len(col) for col in zip(*gather_list_all)]

print('Average Silhouette scores for different number of clusters:\n{}'.format(results_avg))

**Q 1:** Visualize the average Silhouette scores for different number of clusters and different random seeds.

In [ ]:
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')
plt.grid(True)
plt.xticks(np.arange(min(range_n_clusters),max(range_n_clusters)+1,1.0))
plt.plot(range_n_clusters, results_avg)
plt.show()

**Q 2:** Re-fit with the optimal number of clusters identified above. How does the result differ from using _k=3_ earlier in the project?

In [ ]:
# Therefore we choose number of clusters equal to 9

clusterer = KMeans(n_clusters=9, random_state=SEED)
cluster_labels = clusterer.fit_predict(data_with_dummies_ready)

data_with_clusters['Cluster_k=9'] = cluster_labels
data_with_clusters

## **Part 5**: 2D interactive visualization using T-SNE

Each observation is now associated with a cluster – that's great! However, we have no intuitive understanding of how similar the clusters are to each other. To find out, we will project our multi-dimensional data down into two dimensions and visualize it together with our clustering results.

PCA is able to grasp only **linear** projection of high-dimensional space into lower dimensions. In many cases, what is more precise in to learn the non-linear manifolds in which the data shows the highest variance along with. There are different techniques to achieve this goal, all with their own pros and cons. t-Distributed Stochastic Neighbor Embedding (T-SNE) is an state of the art technique to achieve this goal. Following example shows the main intuition behind this technique. For more information, you can have a look [here](https://lvdmaaten.github.io/tsne/). To understand the difference better, look at the following toy example.  

In [ ]:
# Inspired from work by Jake Vanderplas -- <vanderplas@astro.washington.edu>

n_points = 1000
X_sample, color = datasets.make_s_curve(n_points, random_state=0)
n_components = 2

# Original space
fig = plt.figure(figsize=(32, 12))
ax = fig.add_subplot(251, projection='3d')
ax.scatter(X_sample[:, 0], X_sample[:, 1], X_sample[:, 2], c=color, cmap=plt.cm.Spectral)
ax.view_init(4, -72)

# PCA space
t0 = time()
pca = PCA(n_components=n_components, random_state=0)
Y_sample = pca.fit_transform(X_sample)
t1 = time()
print("PCA: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(252)
plt.scatter(Y_sample[:, 0], Y_sample[:, 1], c=color, cmap=plt.cm.Spectral)
plt.title("PCA (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

# TSNE space
t0 = time()
tsne = TSNE(n_components=n_components, init='pca', random_state=0)
Y_sample = tsne.fit_transform(X_sample)
t1 = time()
print("t-SNE: %.2g sec" % (t1 - t0))
ax = fig.add_subplot(253)
plt.scatter(Y_sample[:, 0], Y_sample[:, 1], c=color, cmap=plt.cm.Spectral)
plt.title("t-SNE (%.2g sec)" % (t1 - t0))
ax.xaxis.set_major_formatter(NullFormatter())
ax.yaxis.set_major_formatter(NullFormatter())
plt.axis('tight')

plt.show()

In brief, T-SNE works by minimizing the divergence between two probability distribution between pairs, proportional to their distances, one in the original space and the other in the reduced dimension space. Unlike PCA, T-SNE is sensitive to its configuration parameters. Among them, following parameters worth mentioning here:

* **perplexity**: A parameter representing the number of nearest neighbors considered as 'close' in the original space. 
* **early_exaggeration**: Controls how tight natural clusters in the original space are in the embedded space and how much space will be between them. 
* **learning_rate**: The step size in solving optimization problem using gradient descent. it can make your optimization to converge fast or to escape from local minima. Note that different runs of this algorithm with different initial points can lead to different results due to non-convexity of the optimization surface. One might run the algorithm several times with different initial values and choose the one with minimum divergence score.

**Q 1:** Transform our customer data into lower dimensions using the distances from the fitted k-means model.

Hint: Use `clusterer.transform(<data>)` to get the distances of each data point to the 9 centroids.

In [ ]:
X_kmeans_distances = clusterer.transform(data_with_dummies_ready)
X_kmeans_distances

**Q 2:** Apply T-SNE to the distance matrix and transform the data to 2D space. What's the shape of the transformed data and why?

In [ ]:
tsne2 = TSNE(n_components=2, verbose=1, random_state=1, method='exact')
X_kmeans_distances_tsne2 = tsne2.fit_transform(X_kmeans_distances)

In [ ]:
X_kmeans_distances_tsne2.shape

**Q 2:** Visualize the 2D data. Color the points according to what cluster they belong to. How are the clusters related to each other? Does that make sense?

This question does not require writing any code. Focus on interpreting the visualization and the steps we went through to get there. 

In [ ]:
# Define a colormap of random colors
colormap = np.array(["#6d8dca", "#69de53", "#723bca", "#c3e14c", "#c84dc9", "#68af4e", "#6e6cd5",
"#e3be38", "#4e2d7c", "#5fdfa8", "#d34690", "#3f6d31", "#d44427", "#7fcdd8", "#cb4053", "#5e9981",
"#803a62", "#9b9e39", "#c88cca", "#e1c37b", "#34223b", "#bdd8a3", "#6e3326", "#cfbdce", "#d07d3c",
"#52697d", "#7d6d33", "#d27c88", "#36422b", "#b68f79"])

In [ ]:
# We transform the 2 dimensioanl tsne data into a data frame with associated cluster number and color

def calculate_color(cluster):
    color = colormap[cluster]
    return color

dataset_kmeans_vis = pd.DataFrame(X_kmeans_distances_tsne2, columns=['x', 'y'])
dataset_kmeans_vis['cluster'] = cluster_labels
dataset_kmeans_vis['color'] = dataset_kmeans_vis.cluster.apply(calculate_color)
dataset_kmeans_vis['channel'] = data['Channel']
dataset_kmeans_vis['region'] = data['Region']

dataset_kmeans_vis.head(2)

In [ ]:
# Visualize using bokeh library which is used for interactive visualization
# BokehJS might return an error the first time running this 

source = ColumnDataSource(data=dataset_kmeans_vis)

plot_kmeans = bp.figure(plot_width=800, plot_height=600, title='KMeans clustering of wholesale customers',
    tools='pan,wheel_zoom,box_zoom,reset,hover',
    x_axis_type=None, y_axis_type=None, min_border=1)
output_notebook()
plot_kmeans.scatter(x='x', y='y', color='color', size=5, source=source)
hover = plot_kmeans.select(dict(type=HoverTool))
hover.tooltips=[("index", "$index"),('cluster','@cluster'), ('channel', '@channel'), ('region', '@region')]
show(plot_kmeans)

## **Part 6**: Discussion

* What could be the semantics behind each cluster? 
* How might the set of features in our dataset affect the cluster semantics?